# guppy and hugr to qir conversion and submission to H1 and H2


This example shows how to convert guppy to qir which can be submitted directly to H1 and H2 device, emulator and syntax checker.

You need to install hugr-qir, guppy and pytket-quantinuum for this notebook to work.


In [1]:
# You can write you guppy directly in a notebook or in a separate file
import sys
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)

In [3]:
# Convert hugr to qir

# By default, the function will automatically check the generated QIR to capture most of the issues that could happen.
# this will show an error message with more details about the problem
# the check can be turned off using the keyword argument `validate_qir = False`

from hugr_qir.hugr_to_qir import hugr_to_qir

guppy_qir_bitcode_string = hugr_to_qir(guppy.compile(main))


In [ ]:
# The QIR is normally returned as base64 encoded llvm bitcode.
# To get a human-readable LLVM assemly language string use the keyword argument `emit_text = True`
guppy_qir = hugr_to_qir(main.compile(), emit_text=True)
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1

define void @__hugr__.main.1() #0 {
alloca_block:
  call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* null)
  call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* null)
  call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %0 = call i1 @__quantum__qis__read_result__body(%Result* null)
  call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__mz__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Result* inttoptr (i64 1 to %Result*))
  %1 = call i1 @__quantum__qis__read_result__body(%Result* inttoptr (i64 1 to %Result*))
  %2 = xor i1 %0, %1
  call v

In [4]:
# set up a pytket-quantinnum backend
# This required credentials for the device

from pytket.extensions.quantinuum import QuantinuumBackend, Language

backend = QuantinuumBackend(device_name="H1-1SC")
backend.login()


In [6]:
# Submit the qir to the device (which expects the base64 encoded llvm binary)

h = backend.submit_program(Language.QIR, guppy_qir_bitcode_string, n_shots=10)
r = backend.get_result(h)
shots = r.get_shots()
assert len(shots) == 10
print(h)
print(shots)

('210e11d3993f48aaacd0c51cab682f78', 'null', -1, 'null')
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [6]:
import sys
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    for _ in range(10):
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)

    result("0", measure(q0))
    result("1", measure(q1))



In [8]:
# currently it is not supported to convert guppy containing loops into QIR that is understood by H1 and H2.
# Running this example shows the expected error.

guppy_qir = hugr_to_qir(guppy.compile_module(), validate_qir=True)
